# Sentiment Analysis using Naive Bayes

In this assignment, we will attempt to label tweets with sentiments (positive, neutral and negative) using Naive Bayes classifier. Naive Bayes is a very basic approach to this problem, but gives surprisingly good accuracy sometimes.

**Fill in the Blanks**

## Importing required libraries

In [17]:
import pandas as pd
import re
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
!npx degit Jainu-s/urldata/al -f

npx: installed 1 in 1.295s
> destination directory is not empty. Using --force, continuing
> cloned Jainu-s/urldata#master


## Reading dataset

In [37]:
from google.colab import files
uploaded = files.upload()

import io
data=pd.read_csv(io.BytesIO(uploaded['tweets.csv']), engine='python', skipfooter=1)
data.drop(data.columns[0],axis=1,inplace=True)
data.head()

Saving tweets.csv to tweets (5).csv


,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


## Text processing for the tweets

In [38]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
from nltk.tokenize import word_tokenize
from string import punctuation 
from nltk.corpus import stopwords 

stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
    
def processTweet(tweet):
    # tweet is the text we will pass for preprocessing 
    # convert passed tweet to lower case 
    # print(tweet)
    tweet = tweet.lower()
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
    
    # use work_tokenize imported above to tokenize the tweet
    tweet = word_tokenize(tweet)
    return [word for word in tweet if word not in stopwords]

## Process all tweets

In [43]:
import numpy as np
processed=[]
data.dropna(subset = ["tweets"], inplace=True)
for tweet in data['tweets']:
    # process all tweets using processTweet function above - store in variable 'cleaned' 
    # if np.dtype(tweet) == 'string':
    #   continue
    cleaned=processTweet(tweet)
    processed.append(' '.join(cleaned))

In [48]:
data['processed'] = processed

## Create pipeline and define parameters for GridSearch

In [44]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

## Split data into test and train

In [ ]:
# split data into train and test with split as 0.2 
from sklearn.model_selection import train_test_split
print(data)
X = data['processed']
y = data.labels

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Perform classification (using GridSearch)

In [52]:
# perform GridSearch CV with 10 fold CV using pipeline and tuned_paramters defined above 
clf = GridSearchCV(text_clf, tuned_parameters)
clf.fit(x_train, y_train)
clf.best_params_
clf.score(x_train, y_train)

0.9608735213830755

## Classification report 

In [53]:
# print classification report after predicting on test set with best model obtained in GridSearch
clf.score(x_test, y_test)

0.8436363636363636

## Important:

In [54]:
counts = data.labels.value_counts()
print(counts)

0    941
1    352
2     81
Name: labels, dtype: int64


We can see above that the class distribution is highly imbalanced, this would not lead to good sampling of the data for the classifier. For your learning, try using [SMOTE](https://imbalanced-learn.readthedocs.io/en/stable/api.html) to oversample the minority classes and then evaluate the performance with Naive Bayes and compare.